In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from graphframes import GraphFrame

In [2]:
sparkSession = SparkSession.builder\
                            .appName("spark_sql")\
                            .master('local')\
                            .getOrCreate()

## Create GraphFrame

In [3]:
vertices = sparkSession.createDataFrame([
    ("1", "Alex", 28, "M", "MIPT"),
    ("2", "Emeli", 28, "F", "MIPT"),
    ("3", "Natasha", 27, "F", "MIPT"),
    ("4", "Pavel", 30, "M", "MIPT"),
    ("5", "Oleg", 35, "M", "MIPT"),
    ("6", "Ivan", 30, "M", "MSU"),
    ("7", "Ilya", 29, "M", "MSU")
], ['id', 'name', 'age', 'gemder', 'universty'])

In [4]:
edges = sparkSession.createDataFrame([
    ('1', '2', 'friend'), ('2', '1', 'friend'), 
    ('1', '3', 'friend'), ('3', '1', 'friend'), 
    ('1', '4', 'friend'), ('4', '1', 'friend'), 
    ('2', '3', 'friend'), ('3', '2', 'friend'), 
    ('2', '5', 'friend'), ('5', '2', 'friend'), 
    ('3', '4', 'friend'), ('4', '3', 'friend'), 
    ('3', '5', 'friend'), ('5', '3', 'friend'), 
    ('3', '6', 'friend'), ('6', '3', 'friend'), 
    ('3', '7', 'friend'), ('7', '3', 'friend')
], ['src', 'dst', 'relationship'])

In [5]:
g = GraphFrame(vertices, edges)

## Show data from GraphFrame

In [6]:
g.vertices.show()

+---+-------+---+------+---------+
| id|   name|age|gemder|universty|
+---+-------+---+------+---------+
|  1|   Alex| 28|     M|     MIPT|
|  2|  Emeli| 28|     F|     MIPT|
|  3|Natasha| 27|     F|     MIPT|
|  4|  Pavel| 30|     M|     MIPT|
|  5|   Oleg| 35|     M|     MIPT|
|  6|   Ivan| 30|     M|      MSU|
|  7|   Ilya| 29|     M|      MSU|
+---+-------+---+------+---------+



In [7]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  1|  2|      friend|
|  2|  1|      friend|
|  1|  3|      friend|
|  3|  1|      friend|
|  1|  4|      friend|
|  4|  1|      friend|
|  2|  3|      friend|
|  3|  2|      friend|
|  2|  5|      friend|
|  5|  2|      friend|
|  3|  4|      friend|
|  4|  3|      friend|
|  3|  5|      friend|
|  5|  3|      friend|
|  3|  6|      friend|
|  6|  3|      friend|
|  3|  7|      friend|
|  7|  3|      friend|
+---+---+------------+



In [8]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  7|       1|
|  3|       6|
|  5|       2|
|  6|       1|
|  1|       3|
|  4|       2|
|  2|       3|
+---+--------+



In [9]:
g.outDegrees.show()

+---+---------+
| id|outDegree|
+---+---------+
|  7|        1|
|  3|        6|
|  5|        2|
|  6|        1|
|  1|        3|
|  4|        2|
|  2|        3|
+---+---------+



## Calculate [PageRank](https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html#graphframes.GraphFrame.pageRank)

In [10]:
# Run PageRank until convergence to tolerance "tol"
result = g.pageRank(resetProbability=0.15, tol=0.1)
result.vertices.show()

+---+-------+---+------+---------+------------------+
| id|   name|age|gemder|universty|          pagerank|
+---+-------+---+------+---------+------------------+
|  1|   Alex| 28|     M|     MIPT|1.0932658920083516|
|  3|Natasha| 27|     F|     MIPT|1.9298191847553345|
|  2|  Emeli| 28|     F|     MIPT|1.0932658920083516|
|  4|  Pavel| 30|     M|     MIPT|0.8453189830945632|
|  7|   Ilya| 29|     M|      MSU|0.5965055325194178|
|  6|   Ivan| 30|     M|      MSU|0.5965055325194178|
|  5|   Oleg| 35|     M|     MIPT|0.8453189830945632|
+---+-------+---+------+---------+------------------+



In [11]:
result.edges.show()

+---+---+------------+-------------------+
|src|dst|relationship|             weight|
+---+---+------------+-------------------+
|  1|  2|      friend| 0.3333333333333333|
|  2|  5|      friend| 0.3333333333333333|
|  1|  4|      friend| 0.3333333333333333|
|  6|  3|      friend|                1.0|
|  1|  3|      friend| 0.3333333333333333|
|  3|  7|      friend|0.16666666666666666|
|  3|  2|      friend|0.16666666666666666|
|  3|  6|      friend|0.16666666666666666|
|  4|  1|      friend|                0.5|
|  5|  3|      friend|                0.5|
|  3|  5|      friend|0.16666666666666666|
|  5|  2|      friend|                0.5|
|  7|  3|      friend|                1.0|
|  4|  3|      friend|                0.5|
|  2|  1|      friend| 0.3333333333333333|
|  3|  4|      friend|0.16666666666666666|
|  3|  1|      friend|0.16666666666666666|
|  2|  3|      friend| 0.3333333333333333|
+---+---+------------+-------------------+



In [12]:
# Run PageRank for a fixed number of iterations
result = g.pageRank(resetProbability=0.15, maxIter=10)
result.vertices.show()

+---+-------+---+------+---------+-------------------+
| id|   name|age|gemder|universty|           pagerank|
+---+-------+---+------+---------+-------------------+
|  1|   Alex| 28|     M|     MIPT|  1.120863948063202|
|  3|Natasha| 27|     F|     MIPT|  2.239465678054025|
|  2|  Emeli| 28|     F|     MIPT|  1.120863948063202|
|  4|  Pavel| 30|     M|     MIPT| 0.7882518740579809|
|  7|   Ilya| 29|     M|      MSU|0.47115133885180416|
|  6|   Ivan| 30|     M|      MSU|0.47115133885180416|
|  5|   Oleg| 35|     M|     MIPT| 0.7882518740579809|
+---+-------+---+------+---------+-------------------+



In [13]:
result.edges.show()

+---+---+------------+-------------------+
|src|dst|relationship|             weight|
+---+---+------------+-------------------+
|  1|  2|      friend| 0.3333333333333333|
|  2|  5|      friend| 0.3333333333333333|
|  1|  4|      friend| 0.3333333333333333|
|  6|  3|      friend|                1.0|
|  1|  3|      friend| 0.3333333333333333|
|  3|  7|      friend|0.16666666666666666|
|  3|  2|      friend|0.16666666666666666|
|  3|  6|      friend|0.16666666666666666|
|  4|  1|      friend|                0.5|
|  5|  3|      friend|                0.5|
|  3|  5|      friend|0.16666666666666666|
|  5|  2|      friend|                0.5|
|  7|  3|      friend|                1.0|
|  4|  3|      friend|                0.5|
|  2|  1|      friend| 0.3333333333333333|
|  3|  4|      friend|0.16666666666666666|
|  3|  1|      friend|0.16666666666666666|
|  2|  3|      friend| 0.3333333333333333|
+---+---+------------+-------------------+

